In [1]:
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Age data

In [7]:
acs_age = pd.read_csv("data/ACSST1Y2023.S0101-2024-12-12T233736.csv", thousands=",")

In [8]:
rename_columns_totals = {}
for column in list(acs_age.columns):
    str_split = column.split("!!")
    if len(str_split) == 3:
        if str_split[2] == "Estimate":
            if str_split[1] == "Total":
                rename_columns_totals[column] = str_split[0].strip()
    else:
        continue

In [9]:
states = list(rename_columns_totals.values())
states.append("group")

In [10]:
rename_columns_totals["Label (Grouping)"] = "group"
acs_age_totals = acs_age.rename(columns=rename_columns_totals)

#### Totals Data

In [11]:
acs_age_totals = acs_age_totals[states]

In [12]:
acs_age_totals = acs_age_totals.T.reset_index()
acs_age_totals.columns = acs_age_totals.iloc[52]
acs_age_totals = acs_age_totals.drop(acs_age_totals.index[52:])

In [13]:
acs_age_totals.columns = [
    column.strip().replace(" ", "_") for column in list(acs_age_totals.columns)
]

In [16]:
median_age_df = acs_age_totals[["Median_age_(years)", "group"]]

In [17]:
median_age_df = median_age_df.rename(
    columns={"Median_age_(years)": "median_age", "group": "NAME"}
)

In [22]:
median_age_df["median_age"] = median_age_df["median_age"].astype(float)

## Combine Data

In [23]:
median_age_gdf = states_df.merge(median_age_df, on="NAME", how="left")

In [24]:
median_age_gdf = median_age_gdf[["NAME", "geometry", "median_age"]].dropna()

In [26]:
median_age_gdf = median_age_gdf.to_crs(9311)

In [27]:
median_age_gdf.to_file("data/median_age.gpkg")